<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" width="140px" alt="EPFL_logo">

## Image Processing Laboratory Notebooks
---

This Jupyter Notebook is part of a series of computer laboratories that are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course 
[**MICRO-512 Image Processing II**](https://moodle.epfl.ch/course/view.php?id=463) taught by Prof. M. Unser and Prof. D. Van de Ville.

The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the [Biomedical Imaging Group](http://bigwww.epfl.ch/). 
The distribution or reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2025.

**Authors**: 
    [Pol del Aguila Pla](mailto:pol.delaguilapla@epfl.ch), 
    [Kay Lächler](mailto:kay.lachler@epfl.ch),
    [Alejandro Noguerón Arámburu](mailto:alejandro.nogueronaramburu@epfl.ch),
    [Kamil Seghrouchni](mailto:kamil.seghrouchni@epfl.ch), and
    [Daniel Sage](mailto:daniel.sage@epfl.ch).
    

# Lab 4.2: Orientation
**Released**: Thursday, February 20, 2025

**Submission deadline**: Friday, February 28, 2025, before 23:59 on [Moodle](https://moodle.epfl.ch/course/view.php?id=463)

**Grade weight**: Lab 4 (16 points), 7.5 % of the overall grade

**Help Session**: Thursday, February 27, 2025

**Related lectures**: Chapter 6

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

**Student Name**: 

**SCIPER**:

In [ ]:
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

# Orientation laboratory (13 points)

In this lab, we implement the computation of the structure tensor presented in Chapter 6.2, which can be used to perform directional image analysis.
The block diagram of the complete system is shown in the following flowchart, where $f(x,y)$ is the grayscale input image.

![Drawing](images/block-diagram.png)

Successively, we implement the functions
* `structure_tensor` to generate the structure tensor matrix,
* `orientation_features`, which implements the whole chain of calculations to generate the features needed for directional analysis, and
* `colorize_features` to display the calculated features as a color image.

Then, we use them in two applications relying on directional image analysis,
* a method to select specific orientations, and
* a keypoint detector (Harris corner detector). 

Finally, we discuss an alternative implementation of `structure_tensor`, improving over the previous version.

# Structure tensor (2 points)

To calculate the elements $J_{xx}$, $J_{xy}$ and $J_{yy}$ of the structure tensor, we need a gradient filter and a Gaussian filter.
For the gradient filter, we use [`scipy.ndimage.sobel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.sobel.html) and for the Gaussian filter, we use [`scipy.ndimage.gaussian_filter`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html).

**For 2 points**, implement `structure_tensor` in the cell below. It may be useful to revisit the [figure](#Orientation-laboratory-(13-points)) at the start of this notebook before starting.

In [ ]:
import scipy.ndimage as nd


def structure_tensor(img, sigma):
    Jxx, Jxy, Jyy = [np.empty_like(img) for _ in range(3)]

    # YOUR CODE HERE
    
    return Jxx, Jxy, Jyy

We perform a quick sanity check on a $11 \times 11$ **impulse image** using `sigma=1`. 
You can modify the input image and the sigma value in the cell below to observe the different results.

In [ ]:
%matplotlib widget
import numpy as np
from interactive_kit import imviewer as viewer

size = 11
test_img = np.zeros((size, size))
test_img[size//2, size//2] = 1

Jxx, Jxy, Jyy = structure_tensor(test_img, sigma=1)
view = viewer([test_img, Jxx, Jyy, Jxy], subplots=(2, 2))

Specifically, we check if $J_{xx}$ and $J_{yy}$ are non-negative and $\pi / 2$ rotations of each other, which should be the case for the impulse image.
Then, we check that $J_{xy}$ contains both negative and non-negative numbers, and that all elements that are either in the fifth row or the fifth column of $J_{xy}$ are zero. 

Because the structure tensor is a crucial part of this lab, we will also perform more sophisticated sanity checks comparing your results to our pre-computed correct results.

In [ ]:
import matplotlib.pyplot as plt
import imageio.v3 as imageio

corner, dendrochronology, fingerprint, harris_corner, wave_ramp = [
    imageio.imread(f'images/{name}.tif') / 255.
    for name in ['corner', 'dendrochronolgy', 'fingerprint', 'harris-corner', 'wave-ramp']
]

# you can change the image to any of the ones we imported: corner, dendrochronology, fingerprint, harris_corner, or wave_ramp
image = wave_ramp
Jxx, Jxy, Jyy = structure_tensor(image, sigma=1)
plt.close('all')
view = viewer([image, Jxx, Jxy, Jyy], subplots=(2, 2))

In [ ]:
# Basic sanity checks
test_img = np.zeros((11, 11))
test_img[5, 5] = 1
Jxx, Jxy, Jyy = structure_tensor(test_img, sigma=1)

if not (np.all(Jxx >= 0) and np.all(Jyy >= 0)): 
    print('WARNING!\nJxx and Jyy should be non-negative')
if not np.allclose(np.rot90(Jxx), Jyy):
    print('WARNING!\nJxx should be a rotation of Jyy')
if not np.any(Jxy > 0) and np.any(Jxy < 0):
    print('WARNING!\nJxy should be positive and negative')
if not np.all(abs(Jxy[5, :]) < 1e-5) and np.all(abs(Jxy[:, 5]) < 1e-5):
    print('WARNING!\nFifth row/col should be zero')

# Comparison to pre-computed correct results
Jxx_corr = np.array([
    [0.004, 0.018, 0.033, 0.035, 0.033, 0.018, 0.004],
    [0.025, 0.114, 0.209, 0.224, 0.209, 0.114, 0.025],
    [0.077, 0.35,  0.644, 0.688, 0.644, 0.35,  0.077],
    [0.113, 0.512, 0.942, 1.006, 0.942, 0.512, 0.113],
    [0.077, 0.35,  0.644, 0.688, 0.644, 0.35,  0.077],
    [0.025, 0.114, 0.209, 0.224, 0.209, 0.114, 0.025],
    [0.004, 0.018, 0.033, 0.035, 0.033, 0.018, 0.004]
])
Jyy_corr = Jxx_corr.T
Jxy_corr = np.array([
    [ 0.003,  0.013,  0.019,  0.   , -0.019, -0.013, -0.003],
    [ 0.013,  0.056,  0.082,  0.   , -0.082, -0.056, -0.013],
    [ 0.019,  0.082,  0.119,  0.   , -0.119, -0.082, -0.019],
    [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
    [-0.019, -0.082, -0.119,  0.   ,  0.119,  0.082,  0.019],
    [-0.013, -0.056, -0.082,  0.   ,  0.082,  0.056,  0.013],
    [-0.003, -0.013, -0.019,  0.   ,  0.019,  0.013,  0.003]
])

for yours, reference in zip([Jxx[2:9, 2:9], Jyy[2:9, 2:9], Jxy[2:9, 2:9]], [Jxx_corr, Jyy_corr, Jxy_corr]):
    if not np.allclose(Jxx[2:9, 2:9], Jxx_corr, atol=1e-3): print('Your function does not pass the sanity check')


Now we can apply `structure_tensor` to any of the images (`corner`, `dendrochronology`, `fingerprint`, `harris_corner`, or `wave_ramp`) to inspect the elements of the structure tensor.
In the cell below, you can change the argument to `structure_tensor` to any image and change `sigma` to observe the effect on the result.

# Orientation features

With the structure tensor, we can compute features to help us understand and visualize the orientation of structures in an image.
An easy way to think about the structure tensor is that, for each pixel location `[m,n]`, we can calculate a matrix $\mathbf{J}$ made out of the values of $J_{xx}$, $J_{yy}$, and $J_{xy}$ at that pixel, 

$$
    \mathbf{J}[m,n] = \left[ \begin{array}{cc} J_{xx}[m,n] & J_{xy}[m,n] \\ J_{xy}[m,n] & J_{yy}[m,n]\end{array} \right]\,.
$$

## Feature calculation (4 points)

The following table below shows four features we will compute from the structure tensor (the indices are dropped for simplicity).
There, $\det$ is the determinant and $\operatorname{tr}$ is the trace.

| Feature | Relation to the structure tensor matrix $\mathbf{J}$ |
| :-: | :-: |
| Orientation | $\Theta = \frac{1}{2}\arctan\left(\frac{2J_{xy}}{J_{yy}-J_{xx}}\right)$ |
| Gradient Energy | $E = J_{yy}+J_{xx}$ |
| Coherence | $C = \frac{\sqrt{(J_{yy}-J_{xx})^2+4J_{xy}^2}}{E}$ if $E > 0.01$ else 0 |
| Harris Index | $H = \det(\mathbf{J}) - \kappa \operatorname{tr}(\mathbf{J})^2\mbox{, with }\kappa = 0.05$ |

In the cell below, complete `orientation_features`, implementing the processing chain described in the flowchart at the start of this notebook.
Use `structure_tensor` from the previous section and implement the features (**1 point each**) specified in the table.
Use `np.arctan2` for $\arctan$, as it calculates the correct quadrant.

In [ ]:
def orientation_features(img, sigma, structure_tensor=structure_tensor):
    orientation, energy, coherence, harris = [np.zeros_like(img) for _ in range(4)]

    # YOUR CODE HERE
    
    return orientation, energy, coherence, harris

In [ ]:
# Change the input image and sigma, to see the result on different images: corner, dendrochronology, fingerprint, harris_corner, or wave_ramp
input_img = wave_ramp
sigma = 2
images = [input_img] + list(orientation_features(input_img, sigma=sigma))
# Display the output features
titles = ['Input image', 'Orientation', 'Energy', 'coherence', 'Harris Index']
plt.close('all')
view = viewer(images, title=titles, widgets=True)

As a sanity check, you can run the next four cells to check that each of the output features is in the correct range when applying the function to the `wave_ramp` image using `sigma = 2`.

In [ ]:
# Sanity checks
orientation, energy, coherence, harris = orientation_features(wave_ramp, sigma=2)
if not (abs(orientation.min() + np.pi / 2) < 0.01 and abs(orientation.max() - np.pi / 2) < 0.01):
    print('WARNING!\nThe orientation should be in [-pi/2, pi/2].')


In [ ]:
if not (abs(energy.min() - 0.00023718704) < 0.01 and abs(energy.max() - 0.00044379648) < 0.01):
    print('WARNING!\nThe energy should be in [0.00023718704, 0.00044379648].')
    

In [ ]:
if not (abs(coherence.min()) >= 0.0 and abs(coherence.max()) <= 1.0):
    print('WARNING\nThe coherence should be in the range [0, 1].')
    

In [ ]:
if not (abs(harris.min() + -9.831781e-09) < 0.01 and abs(harris.max() - 2.6322816e-08) < 0.01):
    print('WARNING!\nThe Harris index should be in [9.831781e-09, 2.6322816e-08].')
    

## Feature visualization (2 points)

Until now, we used grayscale images to visualize the orientation features, allowing only a single feature to be displayed at once.
However, we often want to visualize several features in the same image, making the visual analysis more intuitive.
We do this by leveraging the **hue, saturation, value (HSV)** color representation depicted below.
We assign the orientation (which is $2\pi$ periodic) to the hue (which is also periodic), the coherence (a value between $0$ and $1$) to the saturation, and the original image to the value, which allows us to see the objects in the image.

![Drawing](images/hsv_color_representation.png)

**For 2 points**, implement `colorize_features`, taking *orientation*, *coherence*, *input image*, and a *mode* (see table below) as arguments. The function should create an HSV image `hsv_image` from the orientation features, which is converted to RGB to display it with the `viewer`.

The function will have two modes:
In both modes, the orientation $o \in [-\pi/2, \pi/2]$ is mapped to $[0, 1]$ via $o \mapsto (o + \pi/2) / \pi$, which is put into the hue channel.
In mode $0$, the saturation and value channels are left constant at `1`.
In mode $1$, the coherence is put into the saturation channel, and the input image is put into the value channel.
The coherence and the input image can be directly used without any transformation.
See the table below for the specifications of the two modes.  

| Mode | H channel | S channel | V channel |
| :-: | :-: | :-: | :-: |
| 0: Orientation only | orientation | 1 | 1 |
| 1: Features on image | orientation | coherence | input image |

In [ ]:
import matplotlib

def colorize_features(orientation, coherence, img, mode=1):
    # Fill hsv_img[:, :, 0] to set the hue, hsv_img[:, :, 1] to set the saturation, and hsv_img[:, :, 2] to set the value
    hsv = np.zeros((*img.shape, 3))

    # YOUR CODE HERE

    # Convert HSV to RGB
    return matplotlib.colors.hsv_to_rgb(hsv)

Now run the next two cells for a quick test on your function. As usual, remember that these tests are not definitive and that they do not guarantee the full points.

In [ ]:
orientation = np.array([[-np.pi / 2, -np.pi / 4, 0, np.pi / 4, np.pi / 2]])
coherence   = np.array([[0, 65, 130, 195, 255]]) / 255.
img = np.array([[255, 195, 130, 65, 0]]) / 255.
reference = np.array([[[1, 0, 0], [0.5, 1, 0], [0, 1, 1],[0.5,   0, 1], [1,   0,   0]]])

colorized_img = colorize_features(orientation, coherence, img, mode=0)
if not np.allclose(colorized_img, reference):
    print('WARNING!\nYour colorization function is not yet correct for mode 0. Check the comparison below:')
    viewer([ours, reference], title=['Your output', 'Expected output'], subplots=(1,2))
    

In [ ]:
orientation = np.array([[-np.pi / 2, -np.pi / 4, 0, np.pi / 4, np.pi / 2]])
coherence   = np.array([[0, 65, 130, 195, 255]]) / 255.
img = np.array([[255, 195, 130, 65, 0]]) / 255.
reference = np.array([[[1.,         1.,         1.],
  [0.66724337, 0.76470588, 0.56978085],
  [0.24990388, 0.50980392, 0.50980392],
  [0.15743945, 0.05997693, 0.25490196],
  [0.,         0.,         0.,        ]]])

colorized_img = colorize_features(orientation, coherence, img, mode=1)
if not np.allclose(colorized_img, reference):
    print('WARNING!\nYour colorization function is not yet correct for mode 1. Check the comparison below:')
    viewer([ours, reference], title=['Your output', 'Expected output'], subplots=(1,2))


Below we visualize the effect of `colorize_features` function on different images. 
Clicking on `Extra Widgets` reveals the interface to change the mode and sigma, and to apply the colorization by clicking on `Apply Colorization`.
Cycle through the different images by clicking on `Next` and `Prev`.

In [ ]:
from ipywidgets import widgets


mode_dropdown = widgets.Dropdown(options=['0: Orientation only', '1: Features on image'], value='1: Features on image', description='Mode:', disabled=False)
mode_dictionary = {'0: Orientation only': 0, '1: Features on image': 1}
sigma_slider = widgets.IntSlider(value=3, min=1, max=15, step=1, description=r'$\sigma$')
button = widgets.Button(description='Apply Colorization')


def colorization_callback(img):
    mode = mode_dictionary[mode_dropdown.value]
    features = orientation_features(img, sigma=sigma_slider.value)
    return colorize_features(features[0], features[2], img, mode=mode)


plt.close('all')
image_list = [wave_ramp, dendrochronology, fingerprint]
title = ["wave_ramp", "dendrochronology", "fingerprint"]
new_widgets = [mode_dropdown, sigma_slider, button]
view = viewer(image_list, new_widgets=new_widgets, callbacks=[colorization_callback], widgets=True, title=title)

# Application

We implement applications relying on functions implemented in the previous sections, outlining their merit in real-life applications.

## Orientation selection (2 points)

We develop a function that only selects areas of the image with a specific orientation.
In particular, the algorithm should preserve pixels where
 - $E > T E_{max}$, where $E_{max}$ is the maximum energy in the image and $T\in[0, 1]$ is a relative threshold,
 - $C > 0.5$,
 - $\theta_{min} \leq \theta(x, y) \leq \theta_{max}$.

**For 2 points**, implement the function `select_direction` taking the arguments
* `img`: The input image
* `sigma`: $\sigma$ to be used in `orientation_features`
* `T`: The relative energy threshold
* `theta_min`: The minimum angle $\theta_{min}$
* `theta_max`: The maximum angle $\theta_{max}$

and returning

* `output`: Output image keeping the pixels with the given features, with all other pixels set to the minimum value of the image (not necessarily 0).

Use the function `orientation_features` you implemented in [Part 2.A.](#2.A.-Feature-calculation-(4-points)) to get the features needed.

**Note:** We account for the periodicity of $\theta$ as follows:
If $\theta_{\mathrm{min}} \leq \theta_{\mathrm{max}}$ then return the values inside the range $[\theta_{\mathrm{min}}, \theta_{\mathrm{max}}]$, otherwise return the values that are outside this range, i.e., $[-\pi/2,\pi/2] \setminus (\theta_{\mathrm{max}}, \theta_{\mathrm{min}})$ (see [relative complement](https://en.wikipedia.org/wiki/Complement_(set_theory)#Relative_complement)).
As an example, if $\theta_{\mathrm{min}} = \frac{\pi}{3}$ and $\theta_{\mathrm{max}} = -\frac{\pi}{3}$, the function should keep all orientation in the ranges $[\frac{\pi}{3}, \frac{\pi}{2}]$ and $[-\frac{\pi}{2}, -\frac{\pi}{3}]$ but discard all orientations in the range $(-\frac{\pi}{3}, \frac{\pi}{3})$.

In [ ]:
def select_direction(img, sigma, T, theta_min, theta_max):
    assert -np.pi/2 <= theta_min <= np.pi/2, 'theta_min should be in [-pi/2, pi/2]'
    assert -np.pi/2 <= theta_max <= np.pi/2, 'theta_max should be in [-pi/2, pi/2]'
    
    output = img.copy()
    
    # YOUR CODE HERE
    
    return output

The next cell will evaluate your function on a test image that consists of 4 lines at the angles $0$, $\frac{\pi}{4}$, $-\frac{\pi}{4}$ and $\frac{\pi}{2}$. The function will be called on this test image with the ranges $[-\frac{\pi}{6}, \frac{\pi}{6}]$, $[\frac{\pi}{6}, \frac{\pi}{3}]$, $[-\frac{\pi}{3}, -\frac{\pi}{6}]$, and $[\frac{\pi}{3}, -\frac{\pi}{3}]$, which should each extract only one of the lines. Run the cell below to apply this sanity check.

In [ ]:
# Create test image consisting of 4 lines at 0, pi/2, pi/4 and -pi/4
n = 49
r = n//2

line0 = np.zeros((n, n))
line0[r, :r-6] = 1
line0[r, r+7:] = 1
line0 = np.pad(line0, 1)

line90 = np.rot90(line0)

line45 = np.zeros((n, n))
line45[range(n-1, r+6, -1), range(r-6)] = 1
line45[range(r-7, -1, -1), range(r+7, n)] = 1
line45 = np.pad(line45, 1)

lineM45 = np.rot90(line45)

test_img = line0 + line90 + line45 + lineM45

plt.close('all')
view = viewer(test_img)

# Test the 4 lines
lines = [line0, line45, lineM45, line90]
ranges = [[-np.pi/6, np.pi/6], [np.pi/6, np.pi/3], [-np.pi/3, -np.pi/6], [np.pi/3, -np.pi/3]]
names = ['horizontal', 'diagonal ascending', 'diagonal descending', 'vertical']
check = True
for i, ran in enumerate(ranges):
    test_dir = select_direction(test_img, T=0.1, sigma=2, theta_min=ran[0], theta_max=ran[1])
    if not np.allclose(test_dir, lines[i]):
        check = False
        print(f'WARNING!\nOnly the {names[i]} line should be visible at theta_min={ran[0]:.3f}, theta_max={ran[1]:.3f}!\n')
        view = viewer([test_dir, lines[i]], title=[f'Your output for min={ran[0]:.3f}, max={ran[1]:.3f}', 'Expected output'], subplots=(1,2))
if check:
    print('Well done, your function passed the sanity check!')



In the next cell, we apply our function to real images and play around with the different parameters by clicking on the button `Extra Widgets`.
The images can be cycled by clicking *Next* and *Prev*.
For some images, $T$ needs to be very small to extract any orientation.

In [ ]:
# Define Sliders and button
min_slider, max_slider = [
    widgets.FloatSlider(value=-np.pi/2, min=-np.pi/2, max=np.pi/2, step=0.01, description=description)
    for description in ['$\\theta_\\mathrm{min}$', '$\\theta_\\mathrm{max}$']
]
T_slider = widgets.FloatSlider(value=0.5, min=0.05, max=0.95, step=0.05, description=r'$T$')
sigma_slider = widgets.IntSlider(value=3, min=1, max=15, step=1, description=r'$\sigma$')
button = widgets.Button(description='Extract Orientation')


def orientation_callback(img):
    return select_direction(img, sigma_slider.value, T_slider.value, min_slider.value, max_slider.value)


plt.close('all')
image_list = [wave_ramp, dendrochronology, fingerprint]
new_widgets = [sigma_slider, T_slider, min_slider, max_slider, button]
view = viewer(image_list, new_widgets=new_widgets, callbacks=[orientation_callback], widgets=True)

## Harris corner detector (2 points)

The Harris index can be interpreted as the probability of having a corner.
Thus, we can implement a basic corner detector by extracting the local maxima of the Harris index image.

**For 1 point**, complete `detect_corners` taking as arguments
 * `img`: The input image
 * `region_size`: At most one peak is supposed to be detected in a (region_size x region_size) region
 * `threshold`: a relative threshold in the range $[0, 1]$, where only local maxima that are above $T$ times the image maximum are kept. 

and returning
 * `output`: Coordinated of the local maxima.

Get the Harris index using `orientation_features` with `sigma=1`.
To extract the local maxima, use [`skimage.feature.peak_local_max`](https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.peak_local_max).
`min_distance` in `peak_local_max` is related to `region_size` by `region_size = 2 * min_distance + 1`.

In [ ]:
import skimage


def detect_corners(img, region_size, threshold):
    # This is a dummy initialization; the output should be an array of shape (detected_points, 2)
    output = img.copy()
    # YOUR CODE HERE
    return output


def visualize_corners(img, corners):
    peak_mask = np.zeros_like(img)
    peak_mask[tuple(corners.T)] = True
    corners = skimage.morphology.dilation(peak_mask)
    output = np.tile(img[..., None], (1, 1, 3))
    output[corners > 0] = [1, 0, 0]
    return output

Run the next cell to test your `detect_corners` function on a test image that contains 12 corners. Your function should be able to detect them all correctly.

In [ ]:
# Create test image
n = 51
r = n // 2
test_img = np.zeros((n, n))
test_img[r - 15:r + 16,r - 5:r + 6] = 1
test_img[r - 5:r + 6,r - 15:r + 16] = 1

detected_corners = detect_corners(test_img, 3, 0.5)

plt.close('all')
view = viewer(visualize_corners(test_img, detected_corners))
# Check that the number of detected corners is 12
if len(detected_corners) != 12:
    print(f'Warning: Detected {len(detected_corners)} corners instead of 12.')
else:
    print("Correctly detected 12 corners in the image. Verify the correct location by looking at the picture.")


In the cell below, we create an extra widget to experiment with `region_size` and `threshold`.
We use this information to answer the upcoming MCQ.

In [ ]:
T_slider = widgets.FloatSlider(value=0.5, min=0.05, max=0.95, step=0.05, description=r'$T$')
L_slider = widgets.IntSlider(value=3, min=1, max=31, step=2, description=r'$L$')
button = widgets.Button(description='Detect Corners')


def corner_callback(img):
    return visualize_corners(img, detect_corners(img, L_slider.value, T_slider.value))


plt.close('all')
view = viewer([harris_corner, corner], new_widgets=[T_slider, L_slider, button], 
              callbacks=[corner_callback], widgets=True)

### Multiple Choice Question

* Q1: In general, a higher $L$ leads to
    1. more corners, because a higher area is covered,
    2. less corners, because values need to be a local maximum in a larger area,
    3. less corners, because fewer areas of the image are observed, or
    4. more corners, because more areas of the image are observed.


* Q2: In general, selecting a higher $T$ means that a corner has to be ... to be detected.
    1. sharper, so that it's more defined,
    2. rounder, so that it's less defined, or
    3. more diffuse, so that it covers more area.
 
In the next cell, modify the variables `answer_one` and `answer_two` to reflect your answers. The following two cells are for you to check that your answer is in the valid range.

In [ ]:
answer_one = None
answer_two = None
# YOUR CODE HERE

In [ ]:
# Sanity check
if not answer_one in [1, 2, 3, 4]:
    print('WARNING!\nChoose one of 1, 2, 3 or 4.')

In [ ]:
# Sanity check
if not answer_two in [1, 2, 3]:
    print('WARNING!\nChoose one of 1, 2 or 3.')

## *Advanced:* Isotropic filtering in the Fourier space (1 point)

In the first exercise, we used the Sobel filter to compute the gradient for the structure tensor.
We illustrate the *anisotropy* of the Sobel filter using an image that contains the same number of pixels for every possible orientation.

In [ ]:
def create_test_img(ny, nx):
    # Minimum and maximum radial frequencies
    fmin = 0.02
    fmax = 8 * fmin
    # Center
    hx = nx / 2
    hy = ny / 2;
    n = min(nx, ny)
    def structure_func(j, i):
        r = np.sqrt((i - hx)**2 + (j - hy)**2)
        u = 1.0 / (1.0 + np.exp((r - n * 0.45) / 2.0))
        f = fmin + r * (fmax - fmin) / n
        v = np.sin(np.pi * 2 * f * r)
        return (1.0 + v * u) * 128
    return np.fromfunction(structure_func, shape=(ny, nx))


test_img = create_test_img(2048, 2048)
plt.close('all')
view = viewer(test_img)

Plotting the distribution of the orientations (of a circular cut-out) of this image should result in a flat line.
However, the cell below illustrates that this is not the case:

In [ ]:
sigma = 5
orientation = orientation_features(test_img, sigma=sigma)[0]

def plot_orientation_histogram(orientation):
    mask = np.fromfunction(
        lambda i, j: np.sqrt((i-orientation.shape[0]//2)**2 + (j-orientation.shape[1]//2)**2), 
        shape=orientation.shape
    ) <  np.min(orientation.shape)//2 - np.min(orientation.shape)//20
    
    hist, edg = np.histogram(orientation[mask], bins=1000)
    cent = (edg[0:-1] + edg[1:]) / 2
    orientation[~mask] = np.nan  # Only display relevant regions
    plt.close('all')
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    ax[0].imshow(orientation)
    ax[0].set_title('Orientation')
    ax[0].axis('off')
    ax[1].set_title('Orientation distribution')
    ax[1].set_xlabel('Angle in radians')
    ax[1].set_ylabel('Number of pixels')
    ax[1].plot(cent, hist)
    ax[1].grid(); 
    ax[1].set_xticks(ticks=[-np.pi/2,-np.pi/4,0,np.pi/4,np.pi/2], labels=[r'$-\pi/2$',r'$-\pi/4$',r'$0$',r'$\pi/4$',r'$\pi/2$'])
    plt.show()

plot_orientation_histogram(orientation)

The histogram reveals that some orientations are preferred over others.
Specifically, ignoring the peaks at $0$, $\pm\frac{\pi}{4}$ and $\pm\frac{\pi}{2}$ (an artifact of having a limited number of pixels), there are more pixels with an orientation close to $0$ or $\pm\frac{\pi}{2}$ and less pixels close to $\pm\frac{\pi}{4}$. 
This is because the Sobel filters

$$h_x = 
\begin{bmatrix} 
    1 & 0 & -1 \\
    2 & 0 & -2 \\ 
    1 & 0 & -1 
\end{bmatrix}
,\;\;\;\;
h_y = 
\begin{bmatrix} 
    1 & 2 & 1 \\
    0 & 0 & 0 \\ 
    -1 & -2 & -1 
\end{bmatrix}
$$

are anisotropic, favoring grid-aligned edges.
To have a non-biased orientation detector, we re-implement `structure_tensor` using an isotropic gradient filter.
Several choices of these filters exist; in this exercise, we exploit the Fourier property

$$\frac{\partial f(x,y)}{\partial x} \xrightarrow{\mathcal{F}} j\omega_x\operatorname{F}(\omega_x, \omega_y), \;\;\; 
\frac{\partial f(x,y)}{\partial y} \xrightarrow{\mathcal{F}} j\omega_y\operatorname{F}(\omega_x, \omega_y)\,.$$

Thus, to implement an isotropic gradient filter, we need to Fourier transform our image, multiply by $j\omega_x$ and $j\omega_y$ (for the two spatial directions respectively), and invert the Fourier transform.

**For 1 point**, implement `structure_tensor_improved` utilizing the frequency-domain filters $j\omega_x$ and $j\omega_y$.
Use [`np.linspace`](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) to generate the frequency vectors containing equidistant points.
The utility functions `cfft2` and `icfft2` implement the Fourier transform with shifting, such that the DC component is in the center of the image.

In [ ]:
def cfft2(img):
    return np.fft.fftshift(np.fft.fft2(img))


def icfft2(img):
    return np.fft.ifft2(np.fft.ifftshift(img)).real
    

def structure_tensor_improved(img, sigma):
    # Compute the structure tensor using the frequency domain filters
    # Only the gradient computation should change compared to the previous implementation
    # You can reuse parts of the previous implementation
    Jxx, Jxy, Jyy = [np.empty_like(img) for _ in range(3)]

    # YOUR CODE HERE
    
    return Jxx, Jxy, Jyy

We plot the results, comparing them to the previous version using the Sobel gradient filters.
We see that the Fourier version is flat (with the exception of a few sharp spikes), whereas the Sobel version looks like a sinusoidal wave.

In [ ]:
orientation_improved = orientation_features(test_img, sigma=sigma, structure_tensor=structure_tensor_improved)[0]
plot_orientation_histogram(orientation_improved)


🎉 Congratulations on finishing the Orientation lab!! 🎉

Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to Moodle, **in a zip file with the other notebook of this lab.**

* Keep the name of the notebook as: *2_orientation.ipynb*,
* Name the `zip` file: *orientation_lab.zip*.